In [1]:
from IPython import display

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import time
import numpy as np
import sys

In [2]:
def set_figsize(figsize=(3.5,2.5)):
    display.set_matplotlib_formats('svg')
    plt.rcParams['figure.figsize'] = figsize
# 保存在3.11
def semilogy(x_vals, y_vals, x_label, y_label, x2_vals=None, y2_vals=None,
             legend=None, figsize=(3.5, 2.5)):
    set_figsize(figsize)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.semilogy(x_vals, y_vals)
    if x2_vals and y2_vals:
        plt.semilogy(x2_vals, y2_vals, linestyle=':')
        plt.legend(legend)

In [3]:
def sgd(params,lr,batch_size):
    for param in params:
        param.data -= lr*param.grad/batch_size#lr 学习率

In [4]:
def train_ch3(net,train_iter,test_iter,loss,epochs,batch_size,params=None,lr=None,optimizer=None):
    #训练
    for epoch in range(epochs):
        train_l_sum , train_acc_sum,n= 0.0,0.0,0
        for X,y in train_iter:#X为图像
            y_hat=net(X)#对该次输入的预测值
            l = loss(y_hat,y).sum()
            
            #梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            
            l.backward()
            
            if optimizer is None:
                sgd(params,lr,batch_size)
            else:
                optimizer.step()
                
            
            train_l_sum += l.item()#统计总的损失
            train_acc_sum+=(y_hat.argmax(dim=1) == y).sum().item()#统计准确率
            #y.shape[0]为batch_size个label
            n+=y.shape[0]
        #一个epoch走完
        test_acc = evaluate_accuracy(test_iter,net)
        print("Epoch %d, loss %.4f, train accuracy %.3f, test accuracy %.3f" %(epoch+1,train_l_sum/n,train_acc_sum/n,test_acc))
            

In [5]:
mnist_train = torchvision.datasets.FashionMNIST(root='/Users/bob/docs/dataset',train=True,download=False,transform=transforms.ToTensor())

In [6]:
mnist_test = torchvision.datasets.FashionMNIST(root='/Users/bob/docs/dataset/',train=False,download=False,transform=transforms.ToTensor())

In [7]:
def load_data_from_fmnist(batch_size):
    if sys.platform.startswith('win'):
        num_workers = 0
    else:
        num_workers = 4
    train_it = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size,shuffle=True,num_workers=num_workers)
    test_it = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size,shuffle=False,num_workers=num_workers)
    return train_it,test_it

In [8]:
#这也相当于一层？
class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer,self).__init__()
    def forward(self,x):
        return x.view(x.shape[0],-1)

In [2]:
def corr2d(X,K):
    h,w = K.shape
    Y = torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*K).sum()
    return Y